In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torchvision.utils import save_image
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
BATCH_SIZE = 100
LR = 0.0001

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.13, 0.31)])

train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.4MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 494kB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.51MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.42MB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [4]:
class Generator(nn.Module):
    def __init__(self, input_size, output_size):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, output_size),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x).view(-1, 1, 28, 28)


In [5]:
class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x.view(-1, 28 * 28))


In [6]:
# build network
input_size_G = 50
mnist_dim = train_dataset.train_data.size(1) * train_dataset.train_data.size(2)

Generator = Generator(input_size=input_size_G, output_size = mnist_dim).to(device)
Discriminator = Discriminator(mnist_dim).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:76: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [7]:
criterion = nn.BCELoss()
optimizer_G = optim.Adam(Generator.parameters(), lr=LR)
optimizer_D = optim.Adam(Discriminator.parameters(), lr=LR)

In [8]:
def D_train(x):
    Discriminator.zero_grad()

    # train discriminator on real
    x_real, y_real = x.view(-1, mnist_dim), torch.ones(BATCH_SIZE, 1)
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

    D_output = Discriminator(x_real)
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    # train discriminator on fake
    z = Variable(torch.randn(BATCH_SIZE,    input_size_G ).to(device))
    x_fake, y_fake = Generator(z), Variable(torch.zeros(BATCH_SIZE, 1).to(device))

    D_output = Discriminator(x_fake)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    optimizer_D.step()

    return  D_loss.data.item()

In [9]:
def G_train(x):
    Generator.zero_grad()

    z = Variable(torch.randn(BATCH_SIZE, input_size_G).to(device))
    y = Variable(torch.ones(BATCH_SIZE, 1).to(device))

    G_output = Generator(z)
    D_output = Discriminator(G_output)
    G_loss = criterion(D_output, y)

    G_loss.backward()
    optimizer_G.step()

    return G_loss.data.item()

In [10]:
test = Variable(torch.randn(BATCH_SIZE, input_size_G).to(device))
def generate_test_image(epoch):
    with torch.no_grad():
        generated = Generator(test)
        save_image(generated.view(generated.size(0), 1, 28, 28)[0],
                   f'./output/sample_{epoch}.png')

In [12]:
n_epoch = 300
for epoch in range(1, n_epoch + 1):
    D_losses, G_losses = [], []
    with tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch}/{n_epoch}") as pbar:
        for batch_idx, (x, _) in pbar:
            D_losses.append(D_train(x))
            G_losses.append(G_train(x))

            # Обновляем строку с текущими потерями
            pbar.set_postfix({
                "loss_d": torch.mean(torch.FloatTensor(D_losses)).item(),
                "loss_g": torch.mean(torch.FloatTensor(G_losses)).item()
            })

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            epoch, n_epoch,
            torch.mean(torch.FloatTensor(D_losses)),
            torch.mean(torch.FloatTensor(G_losses))))

    generate_test_image(epoch)

Epoch 1/300: 100%|██████████| 600/600 [00:19<00:00, 30.21it/s, loss_d=0.441, loss_g=3.62]


[1/300]: loss_d: 0.441, loss_g: 3.615


Epoch 2/300: 100%|██████████| 600/600 [00:19<00:00, 30.08it/s, loss_d=0.181, loss_g=5.19]


[2/300]: loss_d: 0.181, loss_g: 5.190


Epoch 3/300: 100%|██████████| 600/600 [00:20<00:00, 29.53it/s, loss_d=0.11, loss_g=5.7]


...

[298/300]: loss_d: 0.001, loss_g: 13.396


Epoch 299/300: 100%|██████████| 600/600 [00:20<00:00, 28.59it/s, loss_d=0.00128, loss_g=13.3]


[299/300]: loss_d: 0.001, loss_g: 13.253


Epoch 300/300: 100%|██████████| 600/600 [00:21<00:00, 28.44it/s, loss_d=0.0015, loss_g=13.8]

[300/300]: loss_d: 0.002, loss_g: 13.806


### Попытаемся использовать сверточные слои для улучшения

In [102]:
def generate_test_image(epoch):
    with torch.no_grad():
        test_latent = Variable(torch.randn(BATCH_SIZE, input_size_G, 1, 1).to(device))
        generated = generator(test_latent)
        save_image(generated, f'./output2/sample_{epoch}.png', nrow=10, normalize=True)

In [92]:
# Параметры
input_size_G = 64
mnist_channels = 1  # Для grayscale
image_size = 28
BATCH_SIZE = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [93]:
class Generator(nn.Module):
    def __init__(self, input_size, output_channels=1):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(input_size, 256, kernel_size=7, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.ConvTranspose2d(64, output_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)


In [94]:
class Discriminator(nn.Module):
    def __init__(self, input_channels=1):  # Grayscale input
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x).view(-1, 1)

In [97]:
def D_train(real_images, generator, discriminator, criterion, d_optimizer, device):
    batch_size = real_images.size(0)
    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)

    # 1. Обучение на реальных изображениях
    d_optimizer.zero_grad()
    real_images = real_images.to(device)
    real_outputs = discriminator(real_images)
    real_loss = criterion(real_outputs, real_labels)

    # 2. Обучение на сгенерированных изображениях
    noise = torch.randn(batch_size, generator.model[0].in_channels, 1, 1).to(device)
    fake_images = generator(noise)
    fake_outputs = discriminator(fake_images.detach())
    fake_loss = criterion(fake_outputs, fake_labels)

    # 3. Суммарный лосс и шаг оптимизации
    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()

    return d_loss.item()

In [95]:
def G_train(generator, discriminator, criterion, g_optimizer, device):
    batch_size = BATCH_SIZE
    g_optimizer.zero_grad()

    noise = torch.randn(batch_size, generator.model[0].in_channels, 1, 1).to(device)
    fake_images = generator(noise)
    labels = torch.ones(batch_size, 1).to(device)

    outputs = discriminator(fake_images)
    g_loss = criterion(outputs, labels)

    g_loss.backward()
    g_optimizer.step()

    return g_loss.item()

In [103]:
n_epoch = 100
generator = Generator(input_size=64, output_channels=1).to(device)  # Grayscale

discriminator = Discriminator(input_channels=1).to(device)  # Grayscale

criterion = nn.BCELoss()
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

for epoch in range(1, n_epoch + 1):
    D_losses, G_losses = [], []

    with tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch}/{n_epoch}") as pbar:
        for batch_idx, (x, _) in pbar:
            x = x.to(device)  # Передаем данные на устройство
            d_loss = D_train(x, generator, discriminator, criterion, d_optimizer, device)
            g_loss = G_train(generator, discriminator, criterion, g_optimizer, device)

            D_losses.append(d_loss)
            G_losses.append(g_loss)

            pbar.set_postfix({
                "loss_d": torch.mean(torch.FloatTensor(D_losses)).item(),
                "loss_g": torch.mean(torch.FloatTensor(G_losses)).item()
            })

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            epoch, n_epoch,
            torch.mean(torch.FloatTensor(D_losses)),
            torch.mean(torch.FloatTensor(G_losses))))

    # Генерация тестовых изображений
    generate_test_image(epoch)

Epoch 1/100: 100%|██████████| 600/600 [00:34<00:00, 17.43it/s, loss_d=0.219, loss_g=4.16]


[1/100]: loss_d: 0.219, loss_g: 4.157


Epoch 2/100: 100%|██████████| 600/600 [00:35<00:00, 17.08it/s, loss_d=0.237, loss_g=3.91]


[2/100]: loss_d: 0.237, loss_g: 3.912


Epoch 3/100: 100%|██████████| 600/600 [00:34<00:00, 17.42it/s, loss_d=0.361, loss_g=3.3]


...

[98/100]: loss_d: 0.060, loss_g: 5.029


Epoch 99/100: 100%|██████████| 600/600 [00:35<00:00, 16.73it/s, loss_d=0.457, loss_g=3.75]


[99/100]: loss_d: 0.457, loss_g: 3.746


Epoch 100/100: 100%|██████████| 600/600 [00:35<00:00, 17.01it/s, loss_d=0.332, loss_g=3.43]


[100/100]: loss_d: 0.332, loss_g: 3.429
